In [ ]:
import sys

project_root = os.path.abspath("..")   # sube desde /notebooks a /

if project_root not in sys.path:
    sys.path.append(project_root)

from utils.functions import*

import tensorflow as tf
from keras import layers, models
from keras.utils import to_categorical
import os
import numpy as np
import cv2
import random
from exif import Image


In [2]:
#Comprobamos si nuestras imagenes tienen metadatos para obtener de una amnera facil las etiquetas.

img_path = '../data/train/Letter_A_may_2.png'

with open(img_path, 'rb') as img_file:
    image = Image(img_file)

print(image.has_exif)

False


Como vemos las imagenes de nuestro dataset no tienen metadatos, por lo tanto tendremos que buscar las etiquetas para nuestras neuronas de salida de otra manera.

In [3]:
#Queremos que nuestras imagenes tengan unas dimensiones no muy grandes.
width = 32
height = 32


train_path = '../data/train'
test_path = '../data/test'

train_x = x_generator(train_path, width, height)
train_labels = labels_generator(train_path)
train_y = y_generator(train_path, train_labels)

In [4]:
#Pasamos nuestros datos de train a arrays.

x_data_train = np.array(train_x)
y_data_train = np.array(train_y, dtype=np.dtype(np.float32))

print(x_data_train.shape[0])
print(y_data_train.shape)





35325
(35325, 52)


In [5]:
print('La cantidad de neuronas de salida es de:',len(train_labels))
print(train_labels)

La cantidad de neuronas de salida es de: 52
['A', 'a', 'B', 'b', 'C', 'c', 'D', 'd', 'E', 'e', 'F', 'f', 'G', 'g', 'H', 'h', 'I', 'i', 'J', 'j', 'K', 'k', 'L', 'l', 'M', 'm', 'N', 'n', 'O', 'o', 'P', 'p', 'Q', 'q', 'R', 'r', 'S', 's', 'T', 't', 'U', 'u', 'V', 'v', 'W', 'w', 'X', 'x', 'Y', 'y', 'Z', 'z']


In [6]:
x_data_train = np.array(train_x).reshape(x_data_train.shape[0], width, height, 3)

num_classes = len(train_labels)

#y_data_train = to_categorical(y_data_train) Ya la tenemos categorizada.
print(y_data_train.shape)

(35325, 52)


Ya hemos obtenido nuestras etiquetas.

In [12]:
from sklearn.model_selection import train_test_split

y_indices = np.argmax(y_data_train, axis=1)

X_train, X_val, y_train, y_val = train_test_split(x_data_train, y_data_train, test_size=0.1, random_state=42, stratify=y_indices)

In [23]:


from tensorflow.keras.preprocessing.image import ImageDataGenerator

rango_rotacion = 10
mov_ancho = 0.05
mov_alto = 0.05
rango_acercamiento=[0.9,1.1]

datagen = ImageDataGenerator(
    rotation_range = rango_rotacion,
    width_shift_range = mov_ancho,
    height_shift_range = mov_alto,
    zoom_range=rango_acercamiento,
    rescale=1./225

)


In [24]:
#Los datos para entrenar saldran del datagen, de manera que sean generados con las transformaciones que indicamos
data_gen_entrenamiento = datagen.flow(X_train, y_train, batch_size=32)

In [25]:
val_datagen = ImageDataGenerator(rescale=1./255)

generador_validacion = val_datagen.flow(X_val, y_val, batch_size=32, shuffle=False)

In [9]:
#Normalizamos los datos.

x_data_train_normalized = x_data_train.astype('float32') / 255


In [29]:
model = tf.keras.Sequential([
    layers.Input(shape=(width,height,3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])

In [32]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',   # qué métrica vigilar
    factor=0.5,           # reduce a la mitad la LR
    patience=5,           # espera 5 epochs sin mejora
    min_lr=1e-6,          # límite inferior de LR
    verbose=1
)

In [34]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(data_gen_entrenamiento, 
                    steps_per_epoch=int(np.ceil(len(X_train) / 32)),
                    epochs=80,
                    validation_data=generador_validacion,
                    validation_steps=int(np.ceil(len(X_val) / 32)),
                    callbacks=[reduce_lr])

model.save('model_80_sigmoid_softmax_relu_32_128_callback.keras')

Epoch 1/80
994/994 ━━━━━━━━━━━━━━━━━━━━ 43s 34ms/step - accuracy: 0.8111 - loss: 0.6031 - val_accuracy: 0.8664 - val_loss: 0.4345 - learning_rate: 0.0010
Epoch 2/80
994/994 ━━━━━━━━━━━━━━━━━━━━ 33s 33ms/step - accuracy: 0.8135 - loss: 0.5832 - val_accuracy: 0.8695 - val_loss: 0.4087 - learning_rate: 0.0010
Epoch 3/80
994/994 ━━━━━━━━━━━━━━━━━━━━ 33s 33ms/step - accuracy: 0.8096 - loss: 0.5942 - val_accuracy: 0.8658 - val_loss: 0.4241 - learning_rate: 0.0010
Epoch 4/80
994/994 ━━━━━━━━━━━━━━━━━━━━ 33s 33ms/step - accuracy: 0.8171 - loss: 0.5891 - val_accuracy: 0.8582 - val_loss: 0.4428 - learning_rate: 0.0010
Epoch 5/80
994/994 ━━━━━━━━━━━━━━━━━━━━ 35s 35ms/step - accuracy: 0.8109 - loss: 0.6014 - val_accuracy: 0.8639 - val_loss: 0.4401 - learning_rate: 0.0010
Epoch 6/80
994/994 ━━━━━━━━━━━━━━━━━━━━ 32s 33ms/step - accuracy: 0.8141 - loss: 0.5927 - val_accuracy: 0.8511 - val_loss: 0.4805 - learning_rate: 0.0010
Epoch 7/80
994/994 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8121 - lo

In [49]:
#Predicciones
model_test = models.load_model('model_80_sigmoid_softmax_relu_32_128_callback.keras')

img = cv2.imread('../data/test/Letter_d_min_519.png')

# resize + normalización
img_resized = cv2.resize(img, (32,32))
img_resized = img_resized.astype("float32") / 255.0  

# expandir a lote
img_input = np.expand_dims(img_resized, axis=0)   # (1,32,32,3)

# predicción
result = model_test.predict(img_input)
pred_class = train_labels[np.argmax(result, axis=1)[0]]

print("Vector softmax:", result)
print("Clase predicha:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
Vector softmax: [[1.96532217e-21 1.34480263e-12 1.40861582e-19 5.68195429e-19
  3.21845007e-37 7.48972209e-36 9.46839212e-38 1.00000000e+00
  0.00000000e+00 2.92264004e-22 0.00000000e+00 0.00000000e+00
  5.74689863e-24 1.14929787e-18 1.25889845e-13 2.25410702e-26
  6.99136245e-23 0.00000000e+00 2.46620310e-22 1.69827452e-09
  2.29240371e-31 6.21040929e-35 1.74900227e-27 0.00000000e+00
  8.93544311e-37 3.27254140e-36 1.21267226e-28 0.00000000e+00
  4.52623402e-33 2.94614064e-26 2.21755528e-31 2.64361553e-33
  1.87293973e-17 6.60863510e-17 1.81978224e-30 0.00000000e+00
  2.07789400e-28 4.20653822e-26 0.00000000e+00 2.15010301e-20
  1.86379253e-20 1.59281449e-10 0.00000000e+00 2.13770294e-29
  7.48056073e-27 6.89589507e-17 5.58620032e-37 1.07391276e-29
  0.00000000e+00 3.73074976e-21 0.00000000e+00 1.32268421e-30]]
Clase predicha: d
